<a href="https://colab.research.google.com/github/tasbidrahman10/Research/blob/main/MIMIC_III_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np

base_path = "/content/drive/MyDrive/mimic-iii-clinical-database-demo-1.4"  # folder with ADMISSIONS.csv, ICUSTAYS.csv, CHARTEVENTS.csv


Mounted at /content/drive


In [ ]:
admissions = pd.read_csv(os.path.join(base_path, "ADMISSIONS.csv"))
icustays   = pd.read_csv(os.path.join(base_path, "ICUSTAYS.csv"))
chartevents = pd.read_csv(os.path.join(base_path, "CHARTEVENTS.csv"))


/tmp/ipython-input-2893288858.py:3: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.read_csv(os.path.join(base_path, "CHARTEVENTS.csv"))


In [ ]:
print(icustays.columns)
print(admissions.columns)


Index(['row_id', 'subject_id', 'hadm_id', 'icustay_id', 'dbsource',
       'first_careunit', 'last_careunit', 'first_wardid', 'last_wardid',
       'intime', 'outtime', 'los'],
      dtype='object')
Index(['row_id', 'subject_id', 'hadm_id', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data'],
      dtype='object')


In [ ]:
# Merge ICUSTAYS with ADMISSIONS to get mortality label
icustays["INTIME"] = pd.to_datetime(icustays["intime"])
icustays["OUTTIME"] = pd.to_datetime(icustays["outtime"])

admissions["ADMITTIME"] = pd.to_datetime(admissions["admittime"])
admissions["DISCHTIME"] = pd.to_datetime(admissions["dischtime"])

cohort = icustays.merge(
    admissions[["subject_id", "hadm_id", "hospital_expire_flag"]],
    on=["subject_id", "hadm_id"],
    how="inner"
)

# Mortality label: 1 if died in hospital, else 0
cohort["mortality"] = cohort["hospital_expire_flag"].astype(int)
cohort = cohort[["subject_id", "hadm_id", "icustay_id", "intime", "outtime", "mortality"]]


In [ ]:
d_items = pd.read_csv(os.path.join(base_path, "D_ITEMS.csv"))

print("Heart Rate:")
display(d_items[d_items["label"].str.contains("Heart Rate", case=False, na=False)][["itemid","label","category"]])

print("Systolic BP:")
display(d_items[d_items["label"].str.contains("Systolic", case=False, na=False)][["itemid","label","category"]])

print("Resp Rate:")
display(d_items[d_items["label"].str.contains("Resp", case=False, na=False)][["itemid","label","category"]])

print("O2 Sat:")
display(d_items[d_items["label"].str.contains("O2 Sat", case=False, na=False)][["itemid","label","category"]])

print("Temp:")
display(d_items[d_items["label"].str.contains("Temp", case=False, na=False)][["itemid","label","category"]])
d_items = pd.read_csv(os.path.join(base_path, "D_ITEMS.csv"))



Heart Rate:


,itemid,label,category
211,211,Heart Rate,NaN
1077,3494,Lowest Heart Rate,NaN
9524,220045,Heart Rate,Routine Vital Signs
9525,220046,Heart rate Alarm - High,Alarms
9526,220047,Heart Rate Alarm - Low,Alarms


Systolic BP:


,itemid,label,category
31,6,ABP [Systolic],NaN
56,51,Arterial BP [Systolic],NaN
407,442,Manual BP [Systolic],NaN
418,455,NBP [Systolic],NaN
441,480,Orthostat BP sitting [Systolic],NaN
443,482,OrthostatBP standing [Systolic],NaN
445,484,Orthostatic BP lying [Systolic],NaN
451,492,PAP [Systolic],NaN
617,666,Systolic Unloading,NaN
928,3313,BP Cuff [Systolic],NaN


Resp Rate:


...

Temp:


,itemid,label,category
383,417,Inspired Gas Temp,NaN
390,424,LLE [Temp],NaN
393,427,LUE [Temp],NaN
547,591,RLE [Temperature],NaN
553,597,RUE [Temperature],NaN
...,...,...,...
12048,228242,Pt. Temperature (BG) (SOFT),Labs
12083,228074,Number of attempts to perform (LP),Lumbar Puncture
12097,228090,Number of attempts (THCEN),Thoracentesis
12332,228392,Alsius Bath Temp,Treatments


from matplotlib import pyplot as plt
_df_0['itemid'].plot(kind='hist', bins=20, title='itemid')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['itemid'].plot(kind='line', figsize=(8, 4), title='itemid')
plt.gca().spines[['top', 'right']].set_visible(False)

Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
d_items = pd.read_csv(os.path.join(base_path, "D_ITEMS.csv"))

hr_items  = d_items[d_items["label"].str.contains("Heart Rate",  case=False, na=False)]["itemid"].tolist()
sbp_items = d_items[d_items["label"].str.contains("Systolic",    case=False, na=False)]["itemid"].tolist()
rr_items  = d_items[d_items["label"].str.contains("Resp Rate",   case=False, na=False)]["itemid"].tolist()
spo2_items= d_items[d_items["label"].str.contains("O2 Sat",      case=False, na=False)]["itemid"].tolist()
temp_items= d_items[d_items["label"].str.contains("Temp",        case=False, na=False)]["itemid"].tolist()


VITAL_MAP = {
    "HR": hr_items,
    "SBP": sbp_items,
    "RR": rr_items,
    "SpO2": spo2_items,
    "Temp": temp_items,
}

In [ ]:
# Filter CHARTEVENTS to ICU stays of interest only
ce = chartevents.merge(
    cohort[["icustay_id", "intime"]],
    on="icustay_id",
    how="inner"
)

# Keep only vital ITEMIDs
all_itemids = sum(VITAL_MAP.values(), [])
ce = ce[ce["itemid"].isin(all_itemids)].copy()

# Convert times
ce["charttime"] = pd.to_datetime(ce["charttime"])
ce["intime"] = pd.to_datetime(ce["intime"]) # Convert intime to datetime as well

# Compute hours since ICU admission
ce["hours_from_intime"] = (ce["charttime"] - ce["intime"]).dt.total_seconds() / 3600.0

# Keep only first 24 hours and non-negative times
ce = ce[(ce["hours_from_intime"] >= 0) & (ce["hours_from_intime"] < 24)]

In [ ]:
# Map each ITEMID to a vital name
itemid_to_vital = {}
for vital_name, ids in VITAL_MAP.items():
    for iid in ids:
        itemid_to_vital[iid] = vital_name

ce["vital"] = ce["itemid"].map(itemid_to_vital)

# Discretize to integer hour bins
ce["time_hours"] = ce["hours_from_intime"].astype(int)


In [ ]:
# Use VALUENUM as numeric measurement
ce['value'] = pd.to_numeric(ce['value'], errors='coerce') # Convert 'value' to numeric, coercing errors to NaN
ce = ce[~ce["value"].isna()].copy()

# Group by ICU stay, hour, and vital and take mean
grouped = ce.groupby(["icustay_id", "time_hours", "vital"])["value"].mean().reset_index()

# Pivot to wide: one row per ICU stay + time_hours, columns = vitals
vitals_hourly = grouped.pivot_table(
    index=["icustay_id", "time_hours"],
    columns="vital",
    values="value"
).reset_index()

vitals_hourly.columns.name = None  # remove index name from columns

In [ ]:
vitals_hourly = vitals_hourly.merge(
    cohort[["subject_id", "hadm_id", "icustay_id", "mortality"]],
    on="icustay_id",
    how="left"
)

# Rename columns to match your model expectations
vitals_hourly = vitals_hourly.rename(columns={
    "SUBJECT_ID": "subject_id",
    "HADM_ID": "hadm_id",
    "ICUSTAY_ID": "icustay_id",
})


In [ ]:
# Build full grid of icustay_id x time_hours (0–23)
all_icustays = vitals_hourly["icustay_id"].unique()
full_index = pd.MultiIndex.from_product(
    [all_icustays, np.arange(24)],
    names=["icustay_id", "time_hours"]
)
full = pd.DataFrame(index=full_index).reset_index()

# Merge with vitals and labels
full = full.merge(
    vitals_hourly,
    on=["icustay_id", "time_hours"],
    how="left"
)

In [ ]:
full = full.rename(columns={"icustay_id": "patient_id"})  # if needed

out_path = os.path.join(base_path, "mimic_timeseries.csv")
full.to_csv(out_path, index=False)
print("Saved:", out_path)


Saved: /content/drive/MyDrive/mimic-iii-clinical-database-demo-1.4/mimic_timeseries.csv
